In [67]:
import os
import re
import shutil
import numpy as np
import pandas as pd

In [56]:
train_data = pd.read_csv("D:\\cpgGGCN\\data\\raw\\train.csv")

In [57]:
def clean_code(code):
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
    code = re.sub(r'//.*', '', code)
    code = re.sub(r'\n\s*\n', '\n', code).strip()
    return code

In [63]:
def to_files(code, index, out_path):
    if not os.path.exists(out_path):
        os.makedirs(out_path)

    file_name = f"{index}.c"
    with open(out_path + file_name, 'w') as f:
        f.write(code)

In [ ]:
import subprocess
def joern_parse(input_path, output_path, file_name):
    out_file = file_name + ".bin"
    joern_parse_call = subprocess.run(["joern-parse.bat", input_path, "--output", output_path + out_file],
                                      stdout=subprocess.PIPE, text=True, check=True)
    print(str(joern_parse_call))
    return out_file



In [88]:
# code to CPG.bin
train_codes = []
train_labels = []

SLICE_SIZE = 500
PATH_CODE = "data/code/"
PATH_CPG = "data/cpg/"

for i in range(len(train_data)):
    code = train_data.loc[i, 'code']
    code = clean_code(code)
    train_codes.append(code)
    label = train_data.loc[i, 'Label']
    train_labels.append(label)

train_grouped_codes = [train_codes[i:i + SLICE_SIZE] for i in range(0, len(train_codes), SLICE_SIZE)]
train_grouped_labels = [train_labels[i:i + SLICE_SIZE] for i in range(0, len(train_labels), SLICE_SIZE)]

cpg_files = []
for i in range(int(len(train_codes) / SLICE_SIZE)):
    if os.path.exists(PATH_CODE) and os.path.isdir(PATH_CODE):
        cpg_file = joern_parse(PATH_CODE, PATH_CPG, f"{i}_cpg")
        cpg_files.append(cpg_file)
        print(f"Dataset {i} to cpg.")
        shutil.rmtree(PATH_CODE)

    for j in range(SLICE_SIZE):
        to_files(train_grouped_codes[i][j], i * 500 + j, PATH_CODE)

    shutil.rmtree(PATH_CODE)

In [ ]:
def joern_export(cpg_file_path, output_path, output_format="dot"):
    output_file = output_path + "dot"
    joern_export_call = subprocess.run(
        ["joern-export.bat", cpg_file_path, "-o", output_file, "--format", output_format],
        stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, check=True
    )
    print(f"File xuất ra thành công: {output_file}")
    return output_file